##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 配合使用模板和本地编排器来创建 TFX 流水线

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://tensorflow.google.cn/tfx/tutorials/tfx/template_local">     <img src="https://tensorflow.google.cn/images/tf_logo_32px.png">     在 TensorFlow.org 上查看</a></td>
<td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/tfx/tutorials/tfx/template_beam.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png">在 Google Colab 运行</a> </td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/tfx/tutorials/tfx/template_beam.ipynb"><img width="32px" src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">在 Github 上查看源代码</a></td>
<td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/tfx/tutorials/tfx/template_beam.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png">下载笔记本</a></td>
</table></div>

## 简介

本文档将说明如何使用 TFX Python 软件包中提供的*模板*创建 TensorFlow Extended (TFX) 流水线。其中大部分说明为 Linux shell 命令，还有相应的 Jupyter 笔记本代码单元，它们使用 `!` 调用这些命令。

您将使用芝加哥市发布的 [Taxi Trips 数据集](https://data.cityofchicago.org/Transportation/Taxi-Trips/wrvz-psew)构建流水线。我们强烈建议您尝试使用您自己的数据集构建流水线，并将此流水线用作基准。

我们将构建一个在本地环境下运行的流水线。如果您有兴趣在 Google Cloud 上使用 Kubeflow 编排器，请参阅 <a>Cloud AI Platform Pipelines 上的 TFX 教程</a>。

## 前提条件

- Linux / MacOS
- Python &gt;= 3.5.3

您可以通过[在 Google Colab 上运行此笔记本](https://colab.sandbox.google.com/github/tensorflow/tfx/blob/master/docs/tutorials/tfx/template_local.ipynb)轻松获取所有前提条件。


## 第 1 步：设置环境。

**在整个文档中，我们将展示两次命令。一次作为可直接复制-粘贴的 shell 命令，一次作为 Jupyter 笔记本单元。如果您使用的是 Colab，请直接跳过 shell 脚本块并执行笔记本单元。**

您应准备一个开发环境来构建流水线。

安装 `tfx` Python 软件包。我们建议在本地环境中使用 `virtualenv`。您可以使用以下 shell 脚本代码段来设置您的环境。

```sh
# Create a virtualenv for tfx.
virtualenv -p python3 venv
source venv/bin/activate
# Install python packages.
python -m pip install --upgrade "tfx<2"
```

如果您使用的是 Colab：


In [ ]:
import sys
!{sys.executable} -m pip install --upgrade "tfx<2"

注：软件包安装过程中可能会出现一些错误。例如，

> ERROR: some-package 0.some_version.1 has requirement other-package!=2.0.,&lt;3,&gt;=1.15, but you'll have other-package 2.0.0 which is incompatible.

请暂时忽略这些错误。

In [ ]:
# Set `PATH` to include user python binary directory.
HOME=%env HOME
PATH=%env PATH
%env PATH={PATH}:{HOME}/.local/bin

我们来检查一下 TFX 的版本。

```bash
python -c "from tfx import version ; print('TFX version: {}'.format(version.__version__))"
```

In [ ]:
!python3 -c "from tfx import version ; print('TFX version: {}'.format(version.__version__))"

好了。我们已完成创建流水线的准备工作。

## 第 2 步：将预定义的模板复制到项目目录。

在此步骤中，我们将通过复制预定义模板中的其他文件来创建工作流水线项目目录和文件。

您可以通过更改下面的 `PIPELINE_NAME` 为流水线使用其他名称。这也将成为存放文件的项目目录的名称。

```bash
export PIPELINE_NAME="my_pipeline"
export PROJECT_DIR=~/tfx/${PIPELINE_NAME}
```

In [ ]:
PIPELINE_NAME="my_pipeline"
import os
# Create a project directory under Colab content directory.
PROJECT_DIR=os.path.join(os.sep,"content",PIPELINE_NAME)

TFX 的 TFX Python 软件包中包含 `taxi` 模板。如果您打算解决包括分类和回归在内的逐点预测问题，可以将此模板作为基础。

`tfx template copy` CLI 命令会将预定义的模板文件复制到您的项目目录中。

```sh
tfx template copy \
   --pipeline_name="${PIPELINE_NAME}" \
   --destination_path="${PROJECT_DIR}" \
   --model=taxi
```

In [ ]:
!tfx template copy \
  --pipeline_name={PIPELINE_NAME} \
  --destination_path={PROJECT_DIR} \
  --model=taxi

将此笔记本中的工作目录上下文更改为项目目录。

```bash
cd ${PROJECT_DIR}
```

In [ ]:
%cd {PROJECT_DIR}

## 第 3 步：浏览复制的源文件。

TFX 模板提供了用于构建流水线的基本基架文件，包括 Python 源代码、样本数据，以及用于分析流水线输出的 Jupyter 笔记本。`taxi` 模板使用与 [Airflow 教程](https://tensorflow.google.cn/tfx/tutorials/tfx/airflow_workshop)相同的*芝加哥出租车*数据集和 ML 模型。

在 Google Colab 中，您可以通过点击左侧的文件夹图标浏览文件。文件应该已经复制到项目目录下，在本例中，目录名称为 `my_pipeline`。您可以点击目录名称查看其中的内容，然后双击文件名打开文件。

以下是每个 Python 文件的简要说明。

- `pipeline` - 此目录包含流水线的定义
    - `constants.py` - 定义流水线运行程序的通用常量
    - `pipeline.py` - 定义 TFX 组件和流水线
- `models` - 此目录包含 ML 模型定义
    - `features.py`、`features_test.py` - 定义模型的特征
    - `preprocessing.py`、`preprocessing_test.py` - 使用 `tf::Transform` 定义预处理作业
    - `estimator` - 此目录包含一个基于 Estimator 的模型
        - `constants.py` - 定义模型的常量
        - `model.py`、`model_test.py` - 使用 TF Estimator 定义 DNN 模型
    - `keras` - 此目录包含一个基于 Keras 的模型
        - `constants.py` - 定义模型的常量
        - `model.py` 、`model_test.py` 使用 Keras 定义 DNN 模型
- `local_runner.py`,`kubeflow_dag_runner.py` - 为每个编排引擎定义运行程序


您可能注意到某些文件的名称中包含 `_test.py`。这些是流水线的单元测试，建议您在实现自己的流水线时添加更多单元测试。您可以通过提供带有 `-m` 标记的测试文件的模块名称来运行单元测试。通常，您可以通过删除 `.py` 扩展名并将 `/` 替换为 `.` 来获得模块名称。例如：

```bash
python -m models.features_test
```

In [ ]:
!{sys.executable} -m models.features_test
!{sys.executable} -m models.keras.model_test


## 第 4 步：运行您的第一个 TFX 流水线

您可以使用 `pipeline create` 命令创建流水线。

```bash
tfx pipeline create --engine=local --pipeline_path=local_runner.py
```


In [ ]:
!tfx pipeline create --engine=local --pipeline_path=local_runner.py

然后，您可以使用 `run create` 命令运行创建的流水线。

```sh
tfx run create --engine=local --pipeline_name="${PIPELINE_NAME}"
```

In [ ]:
!tfx run create --engine=local --pipeline_name={PIPELINE_NAME}

如果成功，您将看到 `Component CsvExampleGen is finished`。复制模板时，流水线中仅包含一个 CsvExampleGen 组件。

## 第 5 步：添加用于数据验证的组件。

在此步骤中，您将添加用于数据验证的组件，包括 `StatisticsGen`、`SchemaGen` 和 `ExampleValidator`。如果您对数据验证感兴趣，请参阅 [TensorFlow Data Validation 使用入门](https://tensorflow.google.cn/tfx/data_validation/get_started)。

我们将修改复制到 `pipeline/pipeline.py` 中的流水线定义。如果您使用的是本地环境，可以使用您喜欢的编辑器来编辑文件。如果您使用的是 Google Colab，请进行如下操作：

> **点击左侧的文件夹图标打开 `Files` 视图**。

> **点击打开 `my_pipeline` 目录，点击打开 `pipeline` 目录，然后双击打开 `pipeline.py` 文件**。

> 找到并取消注释将 `StatisticsGen`、`SchemaGen` 和 `ExampleValidator` 添加到流水线的 3 行代码。（提示：查找包含 `TODO(step 5):` 的注释。）

> 您的更改将在几秒钟后自动保存。请确保标签页标题中 `pipeline.py` 前面的 `*` 标记消失。**Colab 中的文件编辑器没有保存按钮或快捷方式。即使在 `playground` 模式下，文件编辑器中的 Python 文件也可以保存到运行时环境。**

现在，您需要使用修改后的流水线定义更新现有流水线。使用 `tfx pipeline update` 命令更新流水线，然后使用 `tfx run create` 命令创建更新后的流水线的新执行运行。

```sh
# Update the pipeline
tfx pipeline update --engine=local --pipeline_path=local_runner.py
# You can run the pipeline the same way.
tfx run create --engine local --pipeline_name "${PIPELINE_NAME}"
```


In [ ]:
# Update the pipeline
!tfx pipeline update --engine=local --pipeline_path=local_runner.py
# You can run the pipeline the same way.
!tfx run create --engine local --pipeline_name {PIPELINE_NAME}

您应该能够看到所添加组件的输出日志。我们的流水线会在 `tfx_pipeline_output/my_pipeline` 目录中创建输出工件。

## 第 6 步：添加用于训练的组件。

在此步骤中，您将添加用于模型训练和模型验证的组件，包括 `Transform`、`Trainer`、`Resolver`、`Evaluator` 和 `Pusher`。

> **打开 `pipeline/pipeline.py`**。找到并取消注释将 `Transform`、`Trainer`、`Resolver`、`Evaluator` 和 `Pusher` 添加到流水线的 5 行代码。（提示：查找 `TODO(step 6):`）

和之前一样，您现在需要使用修改后的流水线定义来更新现有流水线。说明与第 5 步相同。请使用 `tfx pipeline update` 更新流水线，然后使用 `tfx run create` 创建执行运行。

```sh
tfx pipeline update --engine=local --pipeline_path=local_runner.py
tfx run create --engine local --pipeline_name "${PIPELINE_NAME}"
```

In [ ]:
!tfx pipeline update --engine=local --pipeline_path=local_runner.py
!tfx run create --engine local --pipeline_name {PIPELINE_NAME}

在此执行运行成功完成后，您就已经使用本地编排器创建并运行了您的第一个 TFX 流水线！

**注**：您可能已经注意到，每次我们创建流水线运行时，即使输入和参数没有更改，每个组件也会一次又一次地运行。这会浪费时间和资源，您可以使用流水线缓存跳过这些执行。您可以通过为 `pipeline.py` 中的 `Pipeline` 对象指定 `enable_cache=True` 来启用缓存。


## 第 7 步：（*可选*）尝试 BigQueryExampleGen。

[BigQuery] 是一种无服务器、扩缩能力极强且经济实惠的云数据仓库。BigQuery 可在 TFX 中用作训练样本的来源。在此步骤中，我们将向流水线添加 <code>BigQueryExampleGen</code>。

您需要 [Google Cloud Platform](https://cloud.google.com/gcp/getting-started) 帐号来使用 BigQuery。请准备一个 GCP 项目。

使用 colab auth 库或 `gcloud` 实用工具登录您的项目。

```sh
# You need `gcloud` tool to login in local shell environment.
gcloud auth login
```

In [ ]:
if 'google.colab' in sys.modules:
  from google.colab import auth
  auth.authenticate_user()
  print('Authenticated')


您应指定 GCP 项目名称，以使用 TFX 访问 BigQuery 资源。将 `GOOGLE_CLOUD_PROJECT` 环境变量设置为您的项目名称。

```sh
export GOOGLE_CLOUD_PROJECT=YOUR_PROJECT_NAME_HERE
```

In [ ]:
# Set your project name below.
# WARNING! ENTER your project name before running this cell.
%env GOOGLE_CLOUD_PROJECT=YOUR_PROJECT_NAME_HERE

> **打开 `pipeline/pipeline.py`**。注释掉 `CsvExampleGen`，并取消注释创建 `BigQueryExampleGen` 实例的行。您还需要取消注释 `create_pipeline` 函数的 `query` 参数。

我们需要再次指定用于 BigQuery 的 GCP 项目，为此，您需要在创建流水线时在 `beam_pipeline_args` 中设置 `--project`。

> **打开 `configs.py`**。取消注释 `BIG_QUERY_WITH_DIRECT_RUNNER_BEAM_PIPELINE_ARGS` 和 `BIG_QUERY_QUERY` 的定义。您应将此文件中的项目 ID 和区域值替换为用于您的 GCP 项目的正确值。

> **打开 `local_runner.py`**。为 create_pipeline() 方法取消注释 `query` 和 `beam_pipeline_args` 两个参数。

现在，流水线已准备好使用 BigQuery 作为样本源。像在第 5 和第 6 步中那样更新流水线并创建运行。

In [ ]:
!tfx pipeline update --engine=local --pipeline_path=local_runner.py
!tfx run create --engine local --pipeline_name {PIPELINE_NAME}

## 后续步骤：将您的数据提取到流水线中。

我们为使用芝加哥出租车数据集的模型创建了流水线。现在是时候将数据放入流水线中了。

您的数据可以存储在流水线能够访问的任何位置，包括 GCS 或 BigQuery。您将需要修改流水线定义以访问您的数据。

1. 如果您的数据存储在文件中，请修改 `kubeflow_runner.py` 或 `local_runner.py` 中的 `DATA_PATH`，并将其设置为文件的位置。如果您的数据存储在 BigQuery 中，请修改 `pipeline/configs.py` 中的 `BIG_QUERY_QUERY` 以正确查询您的数据。
2. 在 `models/features.py` 中添加特征。
3. 修改 `models/preprocessing.py` 以[转换输入数据用于训练](https://tensorflow.google.cn/tfx/guide/transform)。
4. 修改 `models/keras/model.py` 和 `models/keras/constants.py` 以[描述您的 ML 模型](https://tensorflow.google.cn/tfx/guide/trainer)。
    - 您也可以使用基于 Estimator 的模型。在 `pipeline/configs.py` 中将 `RUN_FN` 常量更改为 `models.estimator.model.run_fn`。

有关更多介绍，请参阅 [Trainer 组件指南](https://tensorflow.google.cn/tfx/guide/trainer)。